In [1]:
# !tar -zxvf /home/user03/dataset/colbert/col/top1000.eval.tar.gz -C /home/user03/dataset/colbert/col/

In [2]:
import collections
import unicodedata
import copy
import json
import math
import re
import six
import tensorflow as tf
import time

import numpy as np
import os

In [3]:
tf.__version__

'1.15.0'

In [4]:
# tokenization.py
def convert_to_bert_input(text, max_seq_length, tokenizer, add_cls):
    tokens = tokenizer.tokenize(text)

    # Account for [CLS] and [SEP] with "- 2"
    if len(tokens) > max_seq_length - 2:
        tokens = tokens[:max_seq_length - 2]

    if add_cls:
        tokens = ["[CLS]"] + tokens
    tokens += ["[SEP]"]

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    return input_ids


def convert_to_unicode(text):
    """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    elif six.PY2:
        if isinstance(text, str):
            return text.decode("utf-8", "ignore")
        elif isinstance(text, unicode):
            return text
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python2 or Python 3?")
def printable_text(text):
    """Returns text encoded in a way suitable for print or `tf.logging`."""

  # These functions want `str` for both Python2 and Python3, but in one case
  # it's a Unicode string and in the other it's a byte string.
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    elif six.PY2:
        if isinstance(text, str):
            return text
        elif isinstance(text, unicode):
            return text.encode("utf-8")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python2 or Python 3?")


def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    index = 0
    with tf.gfile.GFile(vocab_file, "r") as reader:
        while True:
            token = convert_to_unicode(reader.readline())
            if not token:
                break
            token = token.strip()
            vocab[token] = index
            index += 1
    return vocab
def convert_tokens_to_ids(vocab, tokens):
    """Converts a sequence of tokens into ids using the vocab."""
    return [vocab[token] for token in tokens]

def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a peice of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens

In [5]:
class FullTokenizer(object):
    """Runs end-to-end tokenziation."""

    def __init__(self, vocab_file, do_lower_case=True):
        self.vocab = load_vocab(vocab_file)
        self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case)
        self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

    def tokenize(self, text):
        return [
            sub_token
            for token in self.basic_tokenizer.tokenize(text)
            for sub_token in self.wordpiece_tokenizer.tokenize(token)
        ]

    def convert_tokens_to_ids(self, tokens):
        return convert_tokens_to_ids(self.vocab, tokens)

In [6]:
class BasicTokenizer(object):
    """Runs basic tokenization (punctuation splitting, lower casing, etc.)."""

    def __init__(self, do_lower_case=True):
        """Constructs a BasicTokenizer.
        Args:
          do_lower_case: Whether to lower case the input.
        """
        self.do_lower_case = do_lower_case

    def tokenize(self, text):
        """Tokenizes a piece of text."""
        text = convert_to_unicode(text)
        text = self._clean_text(text)
        orig_tokens = whitespace_tokenize(text)
        split_tokens = []
        for token in orig_tokens:
            if self.do_lower_case:
                token = token.lower()
                token = self._run_strip_accents(token)
            split_tokens.extend(self._run_split_on_punc(token))

        output_tokens = whitespace_tokenize(" ".join(split_tokens))
        return output_tokens

    def _run_strip_accents(self, text):
        """Strips accents from a piece of text."""
        text = unicodedata.normalize("NFD", text)
        output = []
        for char in text:
            cat = unicodedata.category(char)
            if cat == "Mn":
                continue
            output.append(char)
        return "".join(output)

    def _run_split_on_punc(self, text):
        """Splits punctuation on a piece of text."""
        chars = list(text)
        i = 0
        start_new_word = True
        output = []
        while i < len(chars):
            char = chars[i]
            if _is_punctuation(char):
                output.append([char])
                start_new_word = True
            else:
                if start_new_word:
                    output.append([])
                start_new_word = False
                output[-1].append(char)
            i += 1
        
        return ["".join(x) for x in output]
    
    def _clean_text(self, text):
        """Performs invalid character removal and whitespace cleanup on text."""
        output = []
        for char in text:
            cp = ord(char)
            if cp == 0 or cp == 0xfffd or _is_control(char):
                continue
            if _is_whitespace(char):
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)

In [7]:
class WordpieceTokenizer(object):
    """Runs WordPiece tokenziation."""

    def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=100):
        self.vocab = vocab
        self.unk_token = unk_token
        self.max_input_chars_per_word = max_input_chars_per_word
    def tokenize(self, text):
        """Tokenizes a piece of text into its word pieces.
    This uses a greedy longest-match-first algorithm to perform tokenization
    using the given vocabulary.
    For example:
      input = "unaffable"
      output = ["un", "##aff", "##able"]
    Args:
      text: A single token or whitespace separated tokens. This should have
        already been passed through `BasicTokenizer.
    Returns:
      A list of wordpiece tokens.
    """

        text = convert_to_unicode(text)

        output_tokens = []
        for token in whitespace_tokenize(text):
            chars = list(token)
            if len(chars) > self.max_input_chars_per_word:
                output_tokens.append(self.unk_token)
                continue

            is_bad = False
            start = 0
            sub_tokens = []
            while start < len(chars):
                end = len(chars)
                cur_substr = None
                while start < end:
                    substr = "".join(chars[start:end])
                    if start > 0:
                        substr = "##" + substr
                    if substr in self.vocab:
                        cur_substr = substr
                        break
                    end -= 1
                if cur_substr is None:
                    is_bad = True
                    break
                sub_tokens.append(cur_substr)
                start = end
            if is_bad:
                output_tokens.append(self.unk_token)
            else:
                output_tokens.extend(sub_tokens)
        return output_tokens
def _is_whitespace(char):
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False

def _is_control(char):
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat.startswith("C"):
        return True
    return False

def _is_punctuation(char):
    cp = ord(char)
    if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
        (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

In [8]:
# convert_msmarco_to_tfrecord.py
flags = tf.flags

FLAGS = flags.FLAGS


flags.DEFINE_string(
    "output_folder", 
    "/home/user03/dataset/colbert/col/bert/",
    "Folder where the tfrecord files will be written.")

flags.DEFINE_string(
    "vocab_file",
    "/home/user03/dataset/colbert/col/bert/uncased_L-12_H-768_A-12/vocab.txt",
    "The vocabulary file that the BERT model was trained on.")

flags.DEFINE_string(
    "train_dataset_path",
    "/home/user03/dataset/colbert/col/triples.train.small.tsv",
    "Path to the MSMARCO training dataset containing the tab separated "
    "<query, positive_paragraph, negative_paragraph> tuples.")

flags.DEFINE_string(
    "dev_dataset_path",
    "/home/user03/dataset/colbert/col/top1000.dev",
    "Path to the MSMARCO training dataset containing the tab separated "
    "<query, positive_paragraph, negative_paragraph> tuples.")

flags.DEFINE_string(
    "eval_dataset_path",
    "/home/user03/dataset/colbert/col/top1000.eval",
    "Path to the MSMARCO eval dataset containing the tab separated "
    "<query, positive_paragraph, negative_paragraph> tuples.")

flags.DEFINE_string(
    "dev_qrels_path",
    "/home/user03/dataset/colbert/col/qrels.dev.small.tsv",
    "Path to the query_id relevant doc ids mapping.")

flags.DEFINE_integer(
    "max_seq_length", 512,
    "The maximum total input sequence length after WordPiece tokenization. "
    "Sequences longer than this will be truncated, and sequences shorter "
    "than this will be padded.")

flags.DEFINE_integer(
    "max_query_length", 64,
    "The maximum query sequence length after WordPiece tokenization. "
    "Sequences longer than this will be truncated.")

flags.DEFINE_integer(
    "num_eval_docs", 1000,
    "The maximum number of docs per query for dev and eval sets.")
flags.DEFINE_string("f", "", "kernel")

In [9]:
def write_to_tf_record(writer, tokenizer, query, docs, labels,
                       ids_file=None, query_id=None, doc_ids=None):
    query = convert_to_unicode(query)
    query_token_ids = convert_to_bert_input(
        text=query, max_seq_length=FLAGS.max_query_length, tokenizer=tokenizer, add_cls=True)

    query_token_ids_tf = tf.train.Feature(
        int64_list=tf.train.Int64List(value=query_token_ids))

    for i, (doc_text, label) in enumerate(zip(docs, labels)):
        
        doc_token_id = convert_to_bert_input(
            text=convert_to_unicode(doc_text),
            max_seq_length=FLAGS.max_seq_length - len(query_token_ids),
            tokenizer=tokenizer,
            add_cls=False)

        doc_ids_tf = tf.train.Feature(
            int64_list=tf.train.Int64List(value=doc_token_id))

        labels_tf = tf.train.Feature(
            int64_list=tf.train.Int64List(value=[label]))

        features = tf.train.Features(feature={
            'query_ids': query_token_ids_tf,
            'doc_ids': doc_ids_tf,
            'label': labels_tf,
        })
        example = tf.train.Example(features=features)
        writer.write(example.SerializeToString())

        if ids_file:
            ids_file.write('\t'.join([query_id, doc_ids[i]]) + '\n')
def convert_eval_dataset(set_name, tokenizer):
    print('Converting {} set to tfrecord...'.format(set_name))
    start_time = time.time()

    if set_name == 'dev':
        dataset_path = FLAGS.dev_dataset_path
        relevant_pairs = set()
        with open(FLAGS.dev_qrels_path) as f:
            for line in f:
                query_id, _, doc_id, _ = line.strip().split('\t')
                relevant_pairs.add('\t'.join([query_id, doc_id]))
    else:
        dataset_path = FLAGS.eval_dataset_path

    queries_docs = collections.defaultdict(list)  
    query_ids = {}
    with open(dataset_path, 'r') as f:
        for i, line in enumerate(f):
            query_id, doc_id, query, doc = line.strip().split('\t')
            label = 0
            if set_name == 'dev':
                if '\t'.join([query_id, doc_id]) in relevant_pairs:
                    label = 1
            queries_docs[query].append((doc_id, doc, label))
            query_ids[query] = query_id

  # Add fake paragraphs to the queries that have less than FLAGS.num_eval_docs.
    queries = list(queries_docs.keys())  # Need to copy keys before iterating.
    for query in queries:
        docs = queries_docs[query]
        docs += max(
            0, FLAGS.num_eval_docs - len(docs)) * [('00000000', 'FAKE DOCUMENT', 0)]
        queries_docs[query] = docs
        
    assert len(
        set(len(docs) == FLAGS.num_eval_docs for docs in queries_docs.values())) == 1, (
        'Not all queries have {} docs'.format(FLAGS.num_eval_docs))

    writer = tf.python_io.TFRecordWriter(
        FLAGS.output_folder + '/dataset_' + set_name + '.tf')

    query_doc_ids_path = (
        FLAGS.output_folder + '/query_doc_ids_' + set_name + '.txt')
    
    with open(query_doc_ids_path, 'w') as ids_file:
        for i, (query, doc_ids_docs) in enumerate(queries_docs.items()):
            doc_ids, docs, labels = zip(*doc_ids_docs)
            query_id = query_ids[query]

            write_to_tf_record(writer=writer,
                         tokenizer=tokenizer,
                         query=query, 
                         docs=docs, 
                         labels=labels,
                         ids_file=ids_file,
                         query_id=query_id,
                         doc_ids=doc_ids)

            if i % 100 == 0:
                print('Writing {} set, query {} of {}'.format(
                    set_name, i, len(queries_docs)))
                time_passed = time.time() - start_time
                hours_remaining = (
                    len(queries_docs) - i) * time_passed / (max(1.0, i) * 3600)
                print('Estimated hours remaining to write the {} set: {}'.format(
                    set_name, hours_remaining))
    writer.close()
def convert_train_dataset(tokenizer):
    print('Converting to Train to tfrecord...')

    start_time = time.time()

    print('Counting number of examples...')
    num_lines = sum(1 for line in open(FLAGS.train_dataset_path, 'r'))
    print('{} examples found.'.format(num_lines))
    writer = tf.python_io.TFRecordWriter(
        FLAGS.output_folder + '/dataset_train.tf')
    
    with open(FLAGS.train_dataset_path, 'r') as f:
        for i, line in enumerate(f):
            if i % 1000 == 0:
                time_passed = int(time.time() - start_time)
                print('Processed training set, line {} of {} in {} sec'.format(i, num_lines, time_passed))
                hours_remaining = (num_lines - i) * time_passed / (max(1.0, i) * 3600)
                print('Estimated hours remaining to write the training set: {}'.format(hours_remaining))
            ### 
            if i == 21796216:
                continue
            query, positive_doc, negative_doc = line.rstrip().split('\t')

            write_to_tf_record(writer=writer,
                         tokenizer=tokenizer,
                         query=query, 
                         docs=[positive_doc, negative_doc], 
                         labels=[1, 0])

    writer.close()
    
def main():
    print('Loading Tokenizer...')
    tokenizer = FullTokenizer(vocab_file=FLAGS.vocab_file, do_lower_case=True)
    
    if not os.path.exists(FLAGS.output_folder):
        os.mkdir(FLAGS.output_folder)

    convert_train_dataset(tokenizer=tokenizer)
    convert_eval_dataset(set_name='dev', tokenizer=tokenizer)
    convert_eval_dataset(set_name='eval', tokenizer=tokenizer)
    print('Done!')  

if __name__ == '__main__':
    main()

Loading Tokenizer...

Converting to Train to tfrecord...
Counting number of examples...
21796217 examples found.

Processed training set, line 0 of 21796217 in 16 sec
Estimated hours remaining to write the training set: 96872.07555555555
Processed training set, line 1000 of 21796217 in 17 sec
Estimated hours remaining to write the training set: 102.92185805555556
Processed training set, line 2000 of 21796217 in 19 sec
Estimated hours remaining to write the training set: 57.512517083333336
Processed training set, line 3000 of 21796217 in 20 sec
Estimated hours remaining to write the training set: 40.35780925925926
Processed training set, line 4000 of 21796217 in 22 sec
Estimated hours remaining to write the training set: 33.29366486111111
Processed training set, line 5000 of 21796217 in 23 sec
Estimated hours remaining to write the training set: 27.844332833333333
Processed training set, line 6000 of 21796217 in 24 sec
Estimated hours remaining to write the training set: 24.211352222222

Processed training set, line 61000 of 21796217 in 101 sec
Estimated hours remaining to write the training set: 9.996616197632058
Processed training set, line 62000 of 21796217 in 102 sec
Estimated hours remaining to write the training set: 9.932303467741935
Processed training set, line 63000 of 21796217 in 103 sec
Estimated hours remaining to write the training set: 9.870023593474427
Processed training set, line 64000 of 21796217 in 105 sec
Estimated hours remaining to write the training set: 9.904005143229167
Processed training set, line 65000 of 21796217 in 106 sec
Estimated hours remaining to write the training set: 9.844055564102565
Processed training set, line 66000 of 21796217 in 108 sec
Estimated hours remaining to write the training set: 9.877371363636364
Processed training set, line 67000 of 21796217 in 109 sec
Estimated hours remaining to write the training set: 9.819588113598673
Processed training set, line 68000 of 21796217 in 110 sec
Estimated hours remaining to write the 

Processed training set, line 125000 of 21796217 in 188 sec
Estimated hours remaining to write the training set: 9.05375288
Processed training set, line 126000 of 21796217 in 189 sec
Estimated hours remaining to write the training set: 9.029257083333333
Processed training set, line 127000 of 21796217 in 191 sec
Estimated hours remaining to write the training set: 9.052538160542433
Processed training set, line 128000 of 21796217 in 192 sec
Estimated hours remaining to write the training set: 9.02842375
Processed training set, line 129000 of 21796217 in 193 sec
Estimated hours remaining to write the training set: 9.00467889965547
Processed training set, line 130000 of 21796217 in 195 sec
Estimated hours remaining to write the training set: 9.027590416666667
Processed training set, line 131000 of 21796217 in 196 sec
Estimated hours remaining to write the training set: 9.004203842239185
Processed training set, line 132000 of 21796217 in 197 sec
Estimated hours remaining to write the trainin

Processed training set, line 189000 of 21796217 in 276 sec
Estimated hours remaining to write the training set: 8.76483229276896
Processed training set, line 190000 of 21796217 in 277 sec
Estimated hours remaining to write the training set: 8.749886124269006
Processed training set, line 191000 of 21796217 in 279 sec
Estimated hours remaining to write the training set: 8.7665147513089
Processed training set, line 192000 of 21796217 in 280 sec
Estimated hours remaining to write the training set: 8.751708275462963
Processed training set, line 193000 of 21796217 in 282 sec
Estimated hours remaining to write the training set: 8.768145069084628
Processed training set, line 194000 of 21796217 in 283 sec
Estimated hours remaining to write the training set: 8.753475674398626
Processed training set, line 195000 of 21796217 in 284 sec
Estimated hours remaining to write the training set: 8.738953886039885
Processed training set, line 196000 of 21796217 in 286 sec
Estimated hours remaining to write

Processed training set, line 253000 of 21796217 in 363 sec
Estimated hours remaining to write the training set: 8.586064746376811
Processed training set, line 254000 of 21796217 in 364 sec
Estimated hours remaining to write the training set: 8.575423215223097
Processed training set, line 255000 of 21796217 in 366 sec
Estimated hours remaining to write the training set: 8.588328346405229
Processed training set, line 256000 of 21796217 in 367 sec
Estimated hours remaining to write the training set: 8.57775568467882
Processed training set, line 257000 of 21796217 in 368 sec
Estimated hours remaining to write the training set: 8.567263138780804
Processed training set, line 258000 of 21796217 in 370 sec
Estimated hours remaining to write the training set: 8.580039071920758
Processed training set, line 259000 of 21796217 in 371 sec
Estimated hours remaining to write the training set: 8.56961337087087
Processed training set, line 260000 of 21796217 in 372 sec
Estimated hours remaining to writ

...

Processed training set, line 21653000 of 21796217 in 29505 sec
Estimated hours remaining to write the training set: 0.05420877765205745
Processed training set, line 21654000 of 21796217 in 29506 sec
Estimated hours remaining to write the training set: 0.05382960810422503
Processed training set, line 21655000 of 21796217 in 29507 sec
Estimated hours remaining to write the training set: 0.053450447920675236
Processed training set, line 21656000 of 21796217 in 29509 sec
Estimated hours remaining to write the training set: 0.0530730956393917
Processed training set, line 21657000 of 21796217 in 29510 sec
Estimated hours remaining to write the training set: 0.052693941271233835
Processed training set, line 21658000 of 21796217 in 29512 sec
Estimated hours remaining to write the training set: 0.05231656898656899
Processed training set, line 21659000 of 21796217 in 29513 sec
Estimated hours remaining to write the training set: 0.05193742043338412
Processed training set, line 21660000 of 217962

Processed training set, line 21714000 of 21796217 in 29588 sec
Estimated hours remaining to write the training set: 0.031119664169557786
Processed training set, line 21715000 of 21796217 in 29590 sec
Estimated hours remaining to write the training set: 0.030741819914549596
Processed training set, line 21716000 of 21796217 in 29591 sec
Estimated hours remaining to write the training set: 0.03036293320593111
Processed training set, line 21717000 of 21796217 in 29593 sec
Estimated hours remaining to write the training set: 0.02998506905752278
Processed training set, line 21718000 of 21796217 in 29594 sec
Estimated hours remaining to write the training set: 0.029606188133754898
Processed training set, line 21719000 of 21796217 in 29595 sec
Estimated hours remaining to write the training set: 0.02922731652009761
Processed training set, line 21720000 of 21796217 in 29597 sec
Estimated hours remaining to write the training set: 0.02884942895692654
Processed training set, line 21721000 of 2179

Processed training set, line 21774000 of 21796217 in 29669 sec
Estimated hours remaining to write the training set: 0.00840906296245267
Processed training set, line 21775000 of 21796217 in 29671 sec
Estimated hours remaining to write the training set: 0.008030738703916317
Processed training set, line 21776000 of 21796217 in 29672 sec
Estimated hours remaining to write the training set: 0.007652140276757286
Processed training set, line 21777000 of 21796217 in 29673 sec
Estimated hours remaining to write the training set: 0.007273551108968178
Processed training set, line 21778000 of 21796217 in 29675 sec
Estimated hours remaining to write the training set: 0.006895203556596361
Processed training set, line 21779000 of 21796217 in 29676 sec
Estimated hours remaining to write the training set: 0.006516620138665687
Processed training set, line 21780000 of 21796217 in 29677 sec
Estimated hours remaining to write the training set: 0.0061380459774512805
Processed training set, line 21781000 of 

Writing dev set, query 4900 of 6980
Estimated hours remaining to write the dev set: 0.36411729290371847
Writing dev set, query 5000 of 6980
Estimated hours remaining to write the dev set: 0.3464763582444191
Writing dev set, query 5100 of 6980
Estimated hours remaining to write the dev set: 0.3287721493581801
Writing dev set, query 5200 of 6980
Estimated hours remaining to write the dev set: 0.31123440721809353
Writing dev set, query 5300 of 6980
Estimated hours remaining to write the dev set: 0.2936514718577547
Writing dev set, query 5400 of 6980
Estimated hours remaining to write the dev set: 0.27617414912299365
Writing dev set, query 5500 of 6980
Estimated hours remaining to write the dev set: 0.2586237506408884
Writing dev set, query 5600 of 6980
Estimated hours remaining to write the dev set: 0.2410438786525102
Writing dev set, query 5700 of 6980
Estimated hours remaining to write the dev set: 0.22344799303497254
Writing dev set, query 5800 of 6980
Estimated hours remaining to writ

Writing eval set, query 5800 of 6837
Estimated hours remaining to write the eval set: 0.18068806822433553
Writing eval set, query 5900 of 6837
Estimated hours remaining to write the eval set: 0.16317742911122612
Writing eval set, query 6000 of 6837
Estimated hours remaining to write the eval set: 0.14565311350554228
Writing eval set, query 6100 of 6837
Estimated hours remaining to write the eval set: 0.1281219147791628
Writing eval set, query 6200 of 6837
Estimated hours remaining to write the eval set: 0.11068237388792004
Writing eval set, query 6300 of 6837
Estimated hours remaining to write the eval set: 0.0932500733869101
Writing eval set, query 6400 of 6837
Estimated hours remaining to write the eval set: 0.07578781204021846
Writing eval set, query 6500 of 6837
Estimated hours remaining to write the eval set: 0.05839649467353128
Writing eval set, query 6600 of 6837
Estimated hours remaining to write the eval set: 0.04095339303468213
Writing eval set, query 6700 of 6837
Estimated h

In [10]:
# modeling
class BertConfig(object):
    """Configuration for `BertModel`."""
    def __init__(self,
               vocab_size,
               hidden_size=768,
               num_hidden_layers=12,
               num_attention_heads=12,
               intermediate_size=3072,
               hidden_act="gelu",
               hidden_dropout_prob=0.1,
               attention_probs_dropout_prob=0.1,
               max_position_embeddings=512,
               type_vocab_size=16,
               initializer_range=0.02):
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.hidden_act = hidden_act
        self.intermediate_size = intermediate_size
        self.hidden_dropout_prob = hidden_dropout_prob
        self.attention_probs_dropout_prob = attention_probs_dropout_prob
        self.max_position_embeddings = max_position_embeddings
        self.type_vocab_size = type_vocab_size
        self.initializer_range = initializer_range

    @classmethod
    def from_dict(cls, json_object):
        """Constructs a `BertConfig` from a Python dictionary of parameters."""
        config = BertConfig(vocab_size=None)
        for (key, value) in six.iteritems(json_object):
            config.__dict__[key] = value
        return config
    @classmethod
    def from_json_file(cls, json_file):
        """Constructs a `BertConfig` from a json file of parameters."""
        with tf.gfile.GFile(json_file, "r") as reader:
            text = reader.read()
        return cls.from_dict(json.loads(text))
    
    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output
    
    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"

In [11]:
class BertModel(object):
    def __init__(self,
               config,
               is_training,
               input_ids,
               input_mask=None,
               token_type_ids=None,
               use_one_hot_embeddings=True,
               scope=None):

        config = copy.deepcopy(config)
        if not is_training:
            config.hidden_dropout_prob = 0.0
            config.attention_probs_dropout_prob = 0.0

        input_shape = get_shape_list(input_ids, expected_rank=2)
        batch_size = input_shape[0]
        seq_length = input_shape[1]

        if input_mask is None:
            input_mask = tf.ones(shape=[batch_size, seq_length], dtype=tf.int32)

        if token_type_ids is None:
            token_type_ids = tf.zeros(shape=[batch_size, seq_length], dtype=tf.int32)

        with tf.variable_scope("bert", scope):
            with tf.variable_scope("embeddings"):
            # Perform embedding lookup on the word ids.
                (self.embedding_output, self.embedding_table) = embedding_lookup(
                    input_ids=input_ids,
                    vocab_size=config.vocab_size,
                    embedding_size=config.hidden_size,
                    initializer_range=config.initializer_range,
                    word_embedding_name="word_embeddings",
                    use_one_hot_embeddings=use_one_hot_embeddings)
            # Add positional embeddings and token type embeddings, then layer
            # normalize and perform dropout.
                self.embedding_output = embedding_postprocessor(
                    input_tensor=self.embedding_output,
                    use_token_type=True,
                    token_type_ids=token_type_ids,
                    token_type_vocab_size=config.type_vocab_size,
                    token_type_embedding_name="token_type_embeddings",
                    use_position_embeddings=True,
                    position_embedding_name="position_embeddings",
                    initializer_range=config.initializer_range,
                    max_position_embeddings=config.max_position_embeddings,
                    dropout_prob=config.hidden_dropout_prob)
            with tf.variable_scope("encoder"):
            # This converts a 2D mask of shape [batch_size, seq_length] to a 3D
            # mask of shape [batch_size, seq_length, seq_length] which is used
            # for the attention scores.
                attention_mask = create_attention_mask_from_input_mask(input_ids, input_mask)

            # Run the stacked transformer.
            # `sequence_output` shape = [batch_size, seq_length, hidden_size].
                self.all_encoder_layers = transformer_model(
                    input_tensor=self.embedding_output,
                    attention_mask=attention_mask,
                    hidden_size=config.hidden_size,
                    num_hidden_layers=config.num_hidden_layers,
                    num_attention_heads=config.num_attention_heads,
                    intermediate_size=config.intermediate_size,
                    intermediate_act_fn=get_activation(config.hidden_act),
                    hidden_dropout_prob=config.hidden_dropout_prob,
                    attention_probs_dropout_prob=config.attention_probs_dropout_prob,
                    initializer_range=config.initializer_range,
                    do_return_all_layers=True)
        
            self.sequence_output = self.all_encoder_layers[-1]
        # The "pooler" converts the encoded sequence tensor of shape
        # [batch_size, seq_length, hidden_size] to a tensor of shape
      # [batch_size, hidden_size]. This is necessary for segment-level
      # (or segment-pair-level) classification tasks where we need a fixed
      # dimensional representation of the segment.
            with tf.variable_scope("pooler"):
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token. We assume that this has been pre-trained
                first_token_tensor = tf.squeeze(self.sequence_output[:, 0:1, :], axis=1)
                self.pooled_output = tf.layers.dense(
                    first_token_tensor,
                    config.hidden_size,
                    activation=tf.tanh,
                    kernel_initializer=create_initializer(config.initializer_range))
    def get_pooled_output(self):
        return self.pooled_output
    
    def get_sequence_output(self):
        return self.sequence_output
    
    def get_all_encoder_layers(self):
        return self.all_encoder_layers

    def get_embedding_output(self):
        return self.embedding_output

    def get_embedding_table(self):
        return self.embedding_table

In [12]:
def gelu(input_tensor):
    cdf = 0.5 * (1.0 + tf.erf(input_tensor / tf.sqrt(2.0)))
    return input_tensor * cdf


def get_activation(activation_string):

  # We assume that anything that"s not a string is already an activation
  # function, so we just return it.
    if not isinstance(activation_string, six.string_types):
        return activation_string
    if not activation_string:
        return None

    act = activation_string.lower()
    if act == "linear":
        return None
    elif act == "relu":
        return tf.nn.relu
    elif act == "gelu":
        return gelu
    elif act == "tanh":
        return tf.tanh
    else:
        raise ValueError("Unsupported activation: %s" % act)

In [13]:
def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match("^(.*):\\d+$", name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if name not in name_to_variable:
            continue
        assignment_map[name] = name
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ":0"] = 1

    return (assignment_map, initialized_variable_names)


def dropout(input_tensor, dropout_prob):

    if dropout_prob is None or dropout_prob == 0.0:
        return input_tensor

    output = tf.nn.dropout(input_tensor, 1.0 - dropout_prob)
    return output


def layer_norm(input_tensor, name=None):
  
    return tf.contrib.layers.layer_norm(inputs=input_tensor, begin_norm_axis=-1, begin_params_axis=-1, scope=name)


def layer_norm_and_dropout(input_tensor, dropout_prob, name=None):
    """Runs layer normalization followed by dropout."""
    output_tensor = layer_norm(input_tensor, name)
    output_tensor = dropout(output_tensor, dropout_prob)
    return output_tensor


def create_initializer(initializer_range=0.02):
    """Creates a `truncated_normal_initializer` with the given range."""
    return tf.truncated_normal_initializer(stddev=initializer_range)


def embedding_lookup(input_ids,
                     vocab_size,
                     embedding_size=128,
                     initializer_range=0.02,
                     word_embedding_name="word_embeddings",
                     use_one_hot_embeddings=False):
    """Looks up words embeddings for id tensor.
  Args:
    input_ids: int32 Tensor of shape [batch_size, seq_length] containing word
      ids.
    vocab_size: int. Size of the embedding vocabulary.
    embedding_size: int. Width of the word embeddings.
    initializer_range: float. Embedding initialization range.
    word_embedding_name: string. Name of the embedding table.
    use_one_hot_embeddings: bool. If True, use one-hot method for word
      embeddings. If False, use `tf.nn.embedding_lookup()`. One hot is better
      for TPUs.
  Returns:
    float Tensor of shape [batch_size, seq_length, embedding_size].
  """
  # This function assumes that the input is of shape [batch_size, seq_length,
  # num_inputs].
  #
  # If the input is a 2D tensor of shape [batch_size, seq_length], we
  # reshape to [batch_size, seq_length, 1].
    if input_ids.shape.ndims == 2:
        input_ids = tf.expand_dims(input_ids, axis=[-1])

    embedding_table = tf.get_variable(
      name=word_embedding_name,
      shape=[vocab_size, embedding_size],
      initializer=create_initializer(initializer_range))

    if use_one_hot_embeddings:
        flat_input_ids = tf.reshape(input_ids, [-1])
        one_hot_input_ids = tf.one_hot(flat_input_ids, depth=vocab_size)
        output = tf.matmul(one_hot_input_ids, embedding_table)
    else:
        output = tf.nn.embedding_lookup(embedding_table, input_ids)

    input_shape = get_shape_list(input_ids)

    output = tf.reshape(output,
                      input_shape[0:-1] + [input_shape[-1] * embedding_size])
    return (output, embedding_table)


def embedding_postprocessor(input_tensor,
                            use_token_type=False,
                            token_type_ids=None,
                            token_type_vocab_size=16,
                            token_type_embedding_name="token_type_embeddings",
                            use_position_embeddings=True,
                            position_embedding_name="position_embeddings",
                            initializer_range=0.02,
                            max_position_embeddings=512,
                            dropout_prob=0.1):
    """Performs various post-processing on a word embedding tensor.
  Args:
    input_tensor: float Tensor of shape [batch_size, seq_length,
      embedding_size].
    use_token_type: bool. Whether to add embeddings for `token_type_ids`.
    token_type_ids: (optional) int32 Tensor of shape [batch_size, seq_length].
      Must be specified if `use_token_type` is True.
    token_type_vocab_size: int. The vocabulary size of `token_type_ids`.
    token_type_embedding_name: string. The name of the embedding table variable
      for token type ids.
    use_position_embeddings: bool. Whether to add position embeddings for the
      position of each token in the sequence.
    position_embedding_name: string. The name of the embedding table variable
      for positional embeddings.
    initializer_range: float. Range of the weight initialization.
    max_position_embeddings: int. Maximum sequence length that might ever be
      used with this model. This can be longer than the sequence length of
      input_tensor, but cannot be shorter.
    dropout_prob: float. Dropout probability applied to the final output tensor.
  Returns:
    float tensor with same shape as `input_tensor`.
  Raises:
    ValueError: One of the tensor shapes or input values is invalid.
  """
    input_shape = get_shape_list(input_tensor, expected_rank=3)
    batch_size = input_shape[0]
    seq_length = input_shape[1]
    width = input_shape[2]

    if seq_length > max_position_embeddings:
        raise ValueError("The seq length (%d) cannot be greater than "
                     "`max_position_embeddings` (%d)" %
                     (seq_length, max_position_embeddings))

    output = input_tensor

    if use_token_type:
        if token_type_ids is None:
            raise ValueError("`token_type_ids` must be specified if"
                       "`use_token_type` is True.")
        token_type_table = tf.get_variable(
            name=token_type_embedding_name,
            shape=[token_type_vocab_size, width],
            initializer=create_initializer(initializer_range))
    # This vocab will be small so we always do one-hot here, since it is always
    # faster for a small vocabulary.
        flat_token_type_ids = tf.reshape(token_type_ids, [-1])
        one_hot_ids = tf.one_hot(flat_token_type_ids, depth=token_type_vocab_size)
        token_type_embeddings = tf.matmul(one_hot_ids, token_type_table)
        token_type_embeddings = tf.reshape(token_type_embeddings,
                                       [batch_size, seq_length, width])
        output += token_type_embeddings

    if use_position_embeddings:
        full_position_embeddings = tf.get_variable(
            name=position_embedding_name,
            shape=[max_position_embeddings, width],
            initializer=create_initializer(initializer_range))
    # Since the position embedding table is a learned variable, we create it
    # using a (long) sequence length `max_position_embeddings`. The actual
    # sequence length might be shorter than this, for faster training of
    # tasks that do not have long sequences.
    #
    # So `full_position_embeddings` is effectively an embedding table
    # for position [0, 1, 2, ..., max_position_embeddings-1], and the current
    # sequence has positions [0, 1, 2, ... seq_length-1], so we can just
    # perform a slice.
        if seq_length < max_position_embeddings:
            position_embeddings = tf.slice(full_position_embeddings, [0, 0],
                                     [seq_length, -1])
        else:
            position_embeddings = full_position_embeddings

        num_dims = len(output.shape.as_list())

    # Only the last two dimensions are relevant (`seq_length` and `width`), so
    # we broadcast among the first dimensions, which is typically just
    # the batch size.
        position_broadcast_shape = []
        for _ in range(num_dims - 2):
            position_broadcast_shape.append(1)
        position_broadcast_shape.extend([seq_length, width])
        position_embeddings = tf.reshape(position_embeddings,
                                     position_broadcast_shape)
        output += position_embeddings

    output = layer_norm_and_dropout(output, dropout_prob)
    return output

In [14]:
def create_attention_mask_from_input_mask(from_tensor, to_mask):
    """Create 3D attention mask from a 2D tensor mask.
  Args:
    from_tensor: 2D or 3D Tensor of shape [batch_size, from_seq_length, ...].
    to_mask: int32 Tensor of shape [batch_size, to_seq_length].
  Returns:
    float Tensor of shape [batch_size, from_seq_length, to_seq_length].
  """
    from_shape = get_shape_list(from_tensor, expected_rank=[2, 3])
    batch_size = from_shape[0]
    from_seq_length = from_shape[1]

    to_shape = get_shape_list(to_mask, expected_rank=2)
    to_seq_length = to_shape[1]

    to_mask = tf.cast(
      tf.reshape(to_mask, [batch_size, 1, to_seq_length]), tf.float32)

  # We don't assume that `from_tensor` is a mask (although it could be). We
  # don't actually care if we attend *from* padding tokens (only *to* padding)
  # tokens so we create a tensor of all ones.
  #
  # `broadcast_ones` = [batch_size, from_seq_length, 1]
    broadcast_ones = tf.ones(
        shape=[batch_size, from_seq_length, 1], dtype=tf.float32)

  # Here we broadcast along two dimensions to create the mask.
    mask = broadcast_ones * to_mask

    return mask


def attention_layer(from_tensor,
                    to_tensor,
                    attention_mask=None,
                    num_attention_heads=1,
                    size_per_head=512,
                    query_act=None,
                    key_act=None,
                    value_act=None,
                    attention_probs_dropout_prob=0.0,
                    initializer_range=0.02,
                    do_return_2d_tensor=False,
                    batch_size=None,
                    from_seq_length=None,
                    to_seq_length=None):
    """Performs multi-headed attention from `from_tensor` to `to_tensor`.
  This is an implementation of multi-headed attention based on "Attention
  is all you Need". If `from_tensor` and `to_tensor` are the same, then
  this is self-attention. Each timestep in `from_tensor` attends to the
  corresponding sequence in `to_tensor`, and returns a fixed-with vector.
  This function first projects `from_tensor` into a "query" tensor and
  `to_tensor` into "key" and "value" tensors. These are (effectively) a list
  of tensors of length `num_attention_heads`, where each tensor is of shape
  [batch_size, seq_length, size_per_head].
  Then, the query and key tensors are dot-producted and scaled. These are
  softmaxed to obtain attention probabilities. The value tensors are then
  interpolated by these probabilities, then concatenated back to a single
  tensor and returned.
  In practice, the multi-headed attention are done with transposes and
  reshapes rather than actual separate tensors.
  Args:
    from_tensor: float Tensor of shape [batch_size, from_seq_length,
      from_width].
    to_tensor: float Tensor of shape [batch_size, to_seq_length, to_width].
    attention_mask: (optional) int32 Tensor of shape [batch_size,
      from_seq_length, to_seq_length]. The values should be 1 or 0. The
      attention scores will effectively be set to -infinity for any positions in
      the mask that are 0, and will be unchanged for positions that are 1.
    num_attention_heads: int. Number of attention heads.
    size_per_head: int. Size of each attention head.
    query_act: (optional) Activation function for the query transform.
    key_act: (optional) Activation function for the key transform.
    value_act: (optional) Activation function for the value transform.
    attention_probs_dropout_prob: (optional) float. Dropout probability of the
      attention probabilities.
    initializer_range: float. Range of the weight initializer.
    do_return_2d_tensor: bool. If True, the output will be of shape [batch_size
      * from_seq_length, num_attention_heads * size_per_head]. If False, the
      output will be of shape [batch_size, from_seq_length, num_attention_heads
      * size_per_head].
    batch_size: (Optional) int. If the input is 2D, this might be the batch size
      of the 3D version of the `from_tensor` and `to_tensor`.
    from_seq_length: (Optional) If the input is 2D, this might be the seq length
      of the 3D version of the `from_tensor`.
    to_seq_length: (Optional) If the input is 2D, this might be the seq length
      of the 3D version of the `to_tensor`.
  Returns:
    float Tensor of shape [batch_size, from_seq_length,
      num_attention_heads * size_per_head]. (If `do_return_2d_tensor` is
      true, this will be of shape [batch_size * from_seq_length,
      num_attention_heads * size_per_head]).
  Raises:
    ValueError: Any of the arguments or tensor shapes are invalid.
  """
    
    def transpose_for_scores(input_tensor, batch_size, num_attention_heads,
                           seq_length, width):
        output_tensor = tf.reshape(input_tensor, [batch_size, seq_length, num_attention_heads, width])

        output_tensor = tf.transpose(output_tensor, [0, 2, 1, 3])
        return output_tensor

    from_shape = get_shape_list(from_tensor, expected_rank=[2, 3])
    to_shape = get_shape_list(to_tensor, expected_rank=[2, 3])

    if len(from_shape) != len(to_shape):
        raise ValueError(
        "The rank of `from_tensor` must match the rank of `to_tensor`.")

    if len(from_shape) == 3:
        batch_size = from_shape[0]
        from_seq_length = from_shape[1]
        to_seq_length = to_shape[1]
    elif len(from_shape) == 2:
        if (batch_size is None or from_seq_length is None or to_seq_length is None):
            raise ValueError(
          "When passing in rank 2 tensors to attention_layer, the values "
          "for `batch_size`, `from_seq_length`, and `to_seq_length` "
          "must all be specified.")

  # Scalar dimensions referenced here:
  #   B = batch size (number of sequences)
  #   F = `from_tensor` sequence length
  #   T = `to_tensor` sequence length
  #   N = `num_attention_heads`
  #   H = `size_per_head`

    from_tensor_2d = reshape_to_matrix(from_tensor)
    to_tensor_2d = reshape_to_matrix(to_tensor)

    # `query_layer` = [B*F, N*H]
    query_layer = tf.layers.dense(
      from_tensor_2d,
      num_attention_heads * size_per_head,
      activation=query_act,
      name="query",
      kernel_initializer=create_initializer(initializer_range))

  # `key_layer` = [B*T, N*H]
    key_layer = tf.layers.dense(
      to_tensor_2d,
      num_attention_heads * size_per_head,
      activation=key_act,
      name="key",
      kernel_initializer=create_initializer(initializer_range))

  # `value_layer` = [B*T, N*H]
    value_layer = tf.layers.dense(
      to_tensor_2d,
      num_attention_heads * size_per_head,
      activation=value_act,
      name="value",
      kernel_initializer=create_initializer(initializer_range))

  # `query_layer` = [B, N, F, H]
    query_layer = transpose_for_scores(query_layer, batch_size,
                                     num_attention_heads, from_seq_length,
                                     size_per_head)

  # `key_layer` = [B, N, T, H]
    key_layer = transpose_for_scores(key_layer, batch_size, num_attention_heads,
                                   to_seq_length, size_per_head)

  # Take the dot product between "query" and "key" to get the raw
  # attention scores.
  # `attention_scores` = [B, N, F, T]
    attention_scores = tf.matmul(query_layer, key_layer, transpose_b=True)
    attention_scores = tf.multiply(attention_scores,
                                 1.0 / math.sqrt(float(size_per_head)))

    if attention_mask is not None:
    # `attention_mask` = [B, 1, F, T]
        attention_mask = tf.expand_dims(attention_mask, axis=[1])

    # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
    # masked positions, this operation will create a tensor which is 0.0 for
    # positions we want to attend and -10000.0 for masked positions.
        adder = (1.0 - tf.cast(attention_mask, tf.float32)) * -10000.0

    # Since we are adding it to the raw scores before the softmax, this is
    # effectively the same as removing these entirely.
        attention_scores += adder

  # Normalize the attention scores to probabilities.
  # `attention_probs` = [B, N, F, T]
    attention_probs = tf.nn.softmax(attention_scores)

  # This is actually dropping out entire tokens to attend to, which might
  # seem a bit unusual, but is taken from the original Transformer paper.
    attention_probs = dropout(attention_probs, attention_probs_dropout_prob)

  # `value_layer` = [B, T, N, H]
    value_layer = tf.reshape(
      value_layer,
      [batch_size, to_seq_length, num_attention_heads, size_per_head])

  # `value_layer` = [B, N, T, H]
    value_layer = tf.transpose(value_layer, [0, 2, 1, 3])

  # `context_layer` = [B, N, F, H]
    context_layer = tf.matmul(attention_probs, value_layer)

  # `context_layer` = [B, F, N, H]
    context_layer = tf.transpose(context_layer, [0, 2, 1, 3])

    if do_return_2d_tensor:
    # `context_layer` = [B*F, N*V]
        context_layer = tf.reshape(
            context_layer,
            [batch_size * from_seq_length, num_attention_heads * size_per_head])
    else:
    # `context_layer` = [B, F, N*V]
        context_layer = tf.reshape(
            context_layer,
            [batch_size, from_seq_length, num_attention_heads * size_per_head])

    return context_layer

In [15]:
def transformer_model(input_tensor,
                      attention_mask=None,
                      hidden_size=768,
                      num_hidden_layers=12,
                      num_attention_heads=12,
                      intermediate_size=3072,
                      intermediate_act_fn=gelu,
                      hidden_dropout_prob=0.1,
                      attention_probs_dropout_prob=0.1,
                      initializer_range=0.02,
                      do_return_all_layers=False):
    if hidden_size % num_attention_heads != 0:
        raise ValueError(
            "The hidden size (%d) is not a multiple of the number of attention "
            "heads (%d)" % (hidden_size, num_attention_heads))

    attention_head_size = int(hidden_size / num_attention_heads)
    input_shape = get_shape_list(input_tensor, expected_rank=3)
    batch_size = input_shape[0]
    seq_length = input_shape[1]
    input_width = input_shape[2]

  # The Transformer performs sum residuals on all layers so the input needs
  # to be the same as the hidden size.
    if input_width != hidden_size:
        raise ValueError("The width of the input tensor (%d) != hidden size (%d)" %
                         (input_width, hidden_size))

  # We keep the representation as a 2D tensor to avoid re-shaping it back and
  # forth from a 3D tensor to a 2D tensor. Re-shapes are normally free on
  # the GPU/CPU but may not be free on the TPU, so we want to minimize them to
  # help the optimizer.
    prev_output = reshape_to_matrix(input_tensor)

    all_layer_outputs = []
    for layer_idx in range(num_hidden_layers):
        with tf.variable_scope("layer_%d" % layer_idx):
            layer_input = prev_output
            
            with tf.variable_scope("attention"):
                attention_heads = []
                with tf.variable_scope("self"):
                    attention_head = attention_layer(
                        from_tensor=layer_input,
                        to_tensor=layer_input,
                        attention_mask=attention_mask,
                        num_attention_heads=num_attention_heads,
                        size_per_head=attention_head_size,
                        attention_probs_dropout_prob=attention_probs_dropout_prob,
                        initializer_range=initializer_range,
                        do_return_2d_tensor=True,
                        batch_size=batch_size,
                        from_seq_length=seq_length,
                        to_seq_length=seq_length)
                    attention_heads.append(attention_head)
                
                attention_output = None
                if len(attention_heads) == 1:
                    attention_output = attention_heads[0]
                else:
                    attention_output = tf.concat(attention_heads, axis=-1)

                with tf.variable_scope("output"):
                    attention_output = tf.layers.dense(
                              attention_output,
                              hidden_size,
                              kernel_initializer=create_initializer(initializer_range))
                    attention_output = dropout(attention_output, hidden_dropout_prob)
                    attention_output = layer_norm(attention_output + layer_input)
                    
            with tf.variable_scope("intermediate"):
                intermediate_output = tf.layers.dense(
                    attention_output,
                    intermediate_size,
                    activation=intermediate_act_fn,
                    kernel_initializer=create_initializer(initializer_range))
            
            with tf.variable_scope("output"):
                layer_output = tf.layers.dense(
                intermediate_output,
                hidden_size,
                kernel_initializer=create_initializer(initializer_range))
            layer_output = dropout(layer_output, hidden_dropout_prob)
            layer_output = layer_norm(layer_output + attention_output)
            prev_output = layer_output
            all_layer_outputs.append(layer_output)
    if do_return_all_layers:
        final_outputs = []
        for layer_output in all_layer_outputs:
            final_output = reshape_from_matrix(layer_output, input_shape)
            final_outputs.append(final_output)
        return final_outputs
    else:
        final_output = reshape_from_matrix(prev_output, input_shape)
        return final_output

In [16]:
def get_shape_list(tensor, expected_rank=None, name=None):
    """Returns a list of the shape of tensor, preferring static dimensions.
  Args:
    tensor: A tf.Tensor object to find the shape of.
    expected_rank: (optional) int. The expected rank of `tensor`. If this is
      specified and the `tensor` has a different rank, and exception will be
      thrown.
    name: Optional name of the tensor for the error message.
  Returns:
    A list of dimensions of the shape of tensor. All static dimensions will
    be returned as python integers, and dynamic dimensions will be returned
    as tf.Tensor scalars.
  """
    if name is None:
        name = tensor.name

    if expected_rank is not None:
        assert_rank(tensor, expected_rank, name)

    shape = tensor.shape.as_list()

    non_static_indexes = []
    for (index, dim) in enumerate(shape):
        if dim is None:
            non_static_indexes.append(index)

    if not non_static_indexes:
        return shape

    dyn_shape = tf.shape(tensor)
    for index in non_static_indexes:
        shape[index] = dyn_shape[index]
    return shape


def reshape_to_matrix(input_tensor):
    """Reshapes a >= rank 2 tensor to a rank 2 tensor (i.e., a matrix)."""
    ndims = input_tensor.shape.ndims
    if ndims < 2:
        raise ValueError("Input tensor must have at least rank 2. Shape = %s" %
                     (input_tensor.shape))
    if ndims == 2:
        return input_tensor

    width = input_tensor.shape[-1]
    output_tensor = tf.reshape(input_tensor, [-1, width])
    return output_tensor


def reshape_from_matrix(output_tensor, orig_shape_list):
    """Reshapes a rank 2 tensor back to its original rank >= 2 tensor."""
    if len(orig_shape_list) == 2:
        return output_tensor

    output_shape = get_shape_list(output_tensor)

    orig_dims = orig_shape_list[0:-1]
    width = output_shape[-1]

    return tf.reshape(output_tensor, orig_dims + [width])


def assert_rank(tensor, expected_rank, name=None):
    """Raises an exception if the tensor rank is not of the expected rank.
  Args:
    tensor: A tf.Tensor to check the rank of.
    expected_rank: Python integer or list of integers, expected rank.
    name: Optional name of the tensor for the error message.
  Raises:
    ValueError: If the expected shape doesn't match the actual shape.
  """
    if name is None:
        name = tensor.name

    expected_rank_dict = {}
    if isinstance(expected_rank, six.integer_types):
        expected_rank_dict[expected_rank] = True
    else:
        for x in expected_rank:
            expected_rank_dict[x] = True
    actual_rank = tensor.shape.ndims
    if actual_rank not in expected_rank_dict:
        scope_name = tf.get_variable_scope().name
        raise ValueError(
            "For the tensor `%s` in scope `%s`, the actual rank "
            "`%d` (shape = %s) is not equal to the expected rank `%s`" %
            (name, scope_name, actual_rank, str(tensor.shape), str(expected_rank)))

In [17]:
# optimizer.py
def create_optimizer(loss, init_lr, num_train_steps, num_warmup_steps, use_tpu):
    """Creates an optimizer training op."""
    global_step = tf.train.get_or_create_global_step()

    learning_rate = tf.constant(value=init_lr, shape=[], dtype=tf.float32)

  # Implements linear decay of the learning rate.
    learning_rate = tf.train.polynomial_decay(
      learning_rate,
      global_step,
      num_train_steps,
      end_learning_rate=0.0,
      power=1.0,
      cycle=False)

  # Implements linear warmup. I.e., if global_step < num_warmup_steps, the
  # learning rate will be `global_step/num_warmup_steps * init_lr`.
    if num_warmup_steps:
        global_steps_int = tf.cast(global_step, tf.int32)
        warmup_steps_int = tf.constant(num_warmup_steps, dtype=tf.int32)

        global_steps_float = tf.cast(global_steps_int, tf.float32)
        warmup_steps_float = tf.cast(warmup_steps_int, tf.float32)

        warmup_percent_done = global_steps_float / warmup_steps_float
        warmup_learning_rate = init_lr * warmup_percent_done

        is_warmup = tf.cast(global_steps_int < warmup_steps_int, tf.float32)
        learning_rate = (
            (1.0 - is_warmup) * learning_rate + is_warmup * warmup_learning_rate)
        
    optimizer = AdamWeightDecayOptimizer(
        learning_rate=learning_rate,
        weight_decay_rate=0.01,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-6,
        exclude_from_weight_decay=["LayerNorm", "layer_norm", "bias"])
    
    if use_tpu:
        optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)
        
    tvars = tf.trainable_variables()
    grads = tf.gradients(loss, tvars)

  # This is how the model was pre-trained.
    (grads, _) = tf.clip_by_global_norm(grads, clip_norm=1.0)

    train_op = optimizer.apply_gradients(
        zip(grads, tvars), global_step=global_step)

    new_global_step = global_step + 1
    train_op = tf.group(train_op, [global_step.assign(new_global_step)])
    return train_op

In [18]:
class AdamWeightDecayOptimizer(tf.train.Optimizer):
    """A basic Adam optimizer that includes "correct" L2 weight decay."""

    def __init__(self,
               learning_rate,
               weight_decay_rate=0.0,
               beta_1=0.9,
               beta_2=0.999,
               epsilon=1e-6,
               exclude_from_weight_decay=None,
               name="AdamWeightDecayOptimizer"):
        """Constructs a AdamWeightDecayOptimizer."""
        super(AdamWeightDecayOptimizer, self).__init__(False, name)

        self.learning_rate = learning_rate
        self.weight_decay_rate = weight_decay_rate
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.epsilon = epsilon
        self.exclude_from_weight_decay = exclude_from_weight_decay

    def apply_gradients(self, grads_and_vars, global_step=None, name=None):
        """See base class."""
        assignments = []
        for (grad, param) in grads_and_vars:
            if grad is None or param is None:
                continue

            param_name = self._get_variable_name(param.name)

            m = tf.get_variable(
              name=param_name + "/adam_m",
              shape=param.shape.as_list(),
              dtype=tf.float32,
              trainable=False,
              initializer=tf.zeros_initializer())
            v = tf.get_variable(
              name=param_name + "/adam_v",
              shape=param.shape.as_list(),
              dtype=tf.float32,
              trainable=False,
              initializer=tf.zeros_initializer())

      # Standard Adam update.
            next_m = (
                tf.multiply(self.beta_1, m) + tf.multiply(1.0 - self.beta_1, grad))
            next_v = (
                tf.multiply(self.beta_2, v) + tf.multiply(1.0 - self.beta_2,
                                                    tf.square(grad)))
            
            update = next_m / (tf.sqrt(next_v) + self.epsilon)
            
            if self._do_use_weight_decay(param_name):
                update += self.weight_decay_rate * param

            update_with_lr = self.learning_rate * update

            next_param = param - update_with_lr

            assignments.extend(
                [param.assign(next_param),
                m.assign(next_m),
                v.assign(next_v)])
        return tf.group(*assignments, name=name)

    def _do_use_weight_decay(self, param_name):
        """Whether to use L2 weight decay for `param_name`."""
        if not self.weight_decay_rate:
            return False
        if self.exclude_from_weight_decay:
            for r in self.exclude_from_weight_decay:
                if re.search(r, param_name) is not None:
                    return False
        return True
    
    def _get_variable_name(self, param_name):
        """Get the variable name from the tensor name."""
        m = re.match("^(.*):\\d+$", param_name)
        if m is not None:
            param_name = m.group(1)
        return param_name

In [19]:
# metrics
def average_precision(gt, pred):
    """
  Computes the average precision.
  This function computes the average prescision at k between two lists of
  items.
  Parameters
  ----------
  gt: set
       A set of ground-truth elements (order doesn't matter)
  pred: list
        A list of predicted elements (order does matter)
  Returns
  -------
  score: double
      The average precision over the input lists
  """

    if not gt:
        return 0.0

    score = 0.0
    num_hits = 0.0
    for i,p in enumerate(pred):
        if p in gt and p not in pred[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    
    return score / max(1.0, len(gt))


def NDCG(gt, pred, use_graded_scores=False):
    score = 0.0
    for rank, item in enumerate(pred):
        if item in gt:
            if use_graded_scores:
                grade = 1.0 / (gt.index(item) + 1)
            else:
                grade = 1.0
            score += grade / np.log2(rank + 2)

    norm = 0.0
    for rank in range(len(gt)):
        if use_graded_scores:
            grade = 1.0 / (rank + 1)
        else:
            grade = 1.0
        norm += grade / np.log2(rank + 2)
    return score / max(0.3, norm)


def metrics(gt, pred, metrics_map):
    '''
  Returns a numpy array containing metrics specified by metrics_map.
  gt: ground-truth items
  pred: predicted items
  '''
    out = np.zeros((len(metrics_map),), np.float32)

    if ('MAP' in metrics_map):
        avg_precision = average_precision(gt=gt, pred=pred)
        out[metrics_map.index('MAP')] = avg_precision

    if ('RPrec' in metrics_map):
        intersec = len(gt & set(pred[:len(gt)]))
        out[metrics_map.index('RPrec')] = intersec / max(1., float(len(gt)))

    if 'MRR' in metrics_map:
        score = 0.0
        for rank, item in enumerate(pred):
            if item in gt:
                score = 1.0 / (rank + 1.0)
                break
        out[metrics_map.index('MRR')] = score

    if 'MRR@10' in metrics_map:
        score = 0.0
        for rank, item in enumerate(pred[:10]):
            if item in gt:
                score = 1.0 / (rank + 1.0)
                break
        out[metrics_map.index('MRR@10')] = score

    if ('NDCG' in metrics_map):
        out[metrics_map.index('NDCG')] = NDCG(gt, pred)

    return out

In [20]:
!ls /home/user03/dataset/colbert/col/bert

dataset_dev.tf	 dataset_train.tf	query_doc_ids_eval.txt
dataset_eval.tf  query_doc_ids_dev.txt	uncased_L-12_H-768_A-12


In [24]:
# run_msmarco
flags = tf.flags

FLAGS = flags.FLAGS
flags.DEFINE_string(
    "data_dir", 
    "/home/user03/dataset/colbert/col/bert",
    "The input data dir. Should contain the .tfrecord files and the supporting "
    "query-docids mapping files.")

flags.DEFINE_string(
    "bert_config_file",
    "/home/user03/dataset/colbert/col/bert/uncased_L-12_H-768_A-12/bert_config.json",
    "The config json file corresponding to the pre-trained BERT model. "
    "This specifies the model architecture.")
flags.DEFINE_string(
    "output_dir", "/home/user03/dataset/colbert/col/bert/traindic/output",
    "The output directory where the model checkpoints will be written.")

flags.DEFINE_boolean(
    "msmarco_output", True,
    "Whether to write the predictions to a MS-MARCO-formatted file.")

flags.DEFINE_string(
    "init_checkpoint",
    "/home/user03/dataset/colbert/col/bert/uncased_L-12_H-768_A-12/bert_model.ckpt",
    "Initial checkpoint (usually from a pre-trained BERT model).")

'''flags.DEFINE_integer(
    "max_seq_length", 512,
    "The maximum total input sequence length after WordPiece tokenization. "
    "Sequences longer than this will be truncated, and sequences shorter "
    "than this will be padded.")'''

flags.DEFINE_bool("do_train", True, "Whether to run training.")

flags.DEFINE_bool("do_eval", True, "Whether to run eval on the dev set.")

flags.DEFINE_integer("train_batch_size", 32, "Total batch size for training.")

flags.DEFINE_integer("eval_batch_size", 32, "Total batch size for eval.")

flags.DEFINE_float("learning_rate", 1e-6, "The initial learning rate for Adam.")

flags.DEFINE_integer("num_train_steps", 400000,
                     "Total number of training steps to perform.")

flags.DEFINE_integer("max_eval_examples", None,
                     "Maximum number of examples to be evaluated.")

flags.DEFINE_integer("num_eval_docs", 1000,
                     "Number of docs per query in the dev and eval files.")

flags.DEFINE_integer(
    "num_warmup_steps", 40000,
    "Number of training steps to perform linear learning rate warmup.")

flags.DEFINE_integer("save_checkpoints_steps", 1000,
                     "How often to save the model checkpoint.")

flags.DEFINE_integer("iterations_per_loop", 1000,
                     "How many steps to make in each estimator call.")

flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")

tf.flags.DEFINE_string(
    "tpu_name", None,
    "The Cloud TPU to use for training. This should be either the name "
    "used when creating the Cloud TPU, or a grpc://ip.address.of.tpu:8470 "
    "url.")

tf.flags.DEFINE_string(
    "tpu_zone", None,
    "[Optional] GCE zone where the Cloud TPU is located in. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

tf.flags.DEFINE_string(
    "gcp_project", None,
    "[Optional] Project name for the Cloud TPU-enabled project. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

tf.flags.DEFINE_string("master", None, "[Optional] TensorFlow master URL.")

flags.DEFINE_integer(
    "num_tpu_cores", 8,
    "Only used if `use_tpu` is True. Total number of TPU cores to use.")

METRICS_MAP = ['MAP', 'RPrec', 'MRR', 'NDCG', 'MRR@10']

DuplicateFlagError: The flag 'bert_config_file' is defined twice. First from /home/user03/anaconda3/envs/bert_rank/lib/python3.7/site-packages/ipykernel_launcher.py, Second from /home/user03/anaconda3/envs/bert_rank/lib/python3.7/site-packages/ipykernel_launcher.py.  Description from first occurrence: The config json file corresponding to the pre-trained BERT model. This specifies the model architecture.

In [ ]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
    """Creates a classification model."""
    model = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)

    output_layer = model.get_pooled_output()
    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)

        return (loss, per_example_loss, log_probs)

In [ ]:
def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument

        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
            tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)
        (total_loss, per_example_loss, log_probs) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)

        tvars = tf.trainable_variables()

        scaffold_fn = None
        initialized_variable_names = []
        if init_checkpoint:
            (assignment_map, initialized_variable_names
            ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:
                
                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:
            
            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                  mode=mode,
                  loss=total_loss,
                  train_op=train_op,
                  scaffold_fn=scaffold_fn)

        elif mode == tf.estimator.ModeKeys.PREDICT:
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode,
                predictions={
                    "log_probs": log_probs,
                    "label_ids": label_ids,
                },
                scaffold_fn=scaffold_fn)

        else:
            raise ValueError("Only TRAIN and PREDICT modes are supported: %s" % (mode))

        return output_spec
    return model_fn

In [ ]:
def input_fn_builder(dataset_path, seq_length, is_training,
                     max_eval_examples=None):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""
    
    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]
        output_buffer_size = batch_size * 1000

        def extract_fn(data_record):
            features = {
                "query_ids": tf.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
                "doc_ids": tf.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
                "label": tf.FixedLenFeature([], tf.int64),
            }
            sample = tf.parse_single_example(data_record, features)

            query_ids = tf.cast(sample["query_ids"], tf.int32)
            doc_ids = tf.cast(sample["doc_ids"], tf.int32)
            label_ids = tf.cast(sample["label"], tf.int32)
            input_ids = tf.concat((query_ids, doc_ids), 0)

            query_segment_id = tf.zeros_like(query_ids)
            doc_segment_id = tf.ones_like(doc_ids)
            segment_ids = tf.concat((query_segment_id, doc_segment_id), 0)

            input_mask = tf.ones_like(input_ids)

            features = {
                "input_ids": input_ids,
                "segment_ids": segment_ids,
                "input_mask": input_mask,
                "label_ids": label_ids,
            }
            return features
        
        dataset = tf.data.TFRecordDataset([dataset_path])
        dataset = dataset.map(
            extract_fn, num_parallel_calls=4).prefetch(output_buffer_size)

        if is_training:
            dataset = dataset.repeat()
            dataset = dataset.shuffle(buffer_size=1000)
        else:
            if max_eval_examples:
            # Use at most this number of examples (debugging only).
                dataset = dataset.take(max_eval_examples)
            # pass

        dataset = dataset.padded_batch(
            batch_size=batch_size,
            padded_shapes={
                "input_ids": [seq_length],
                "segment_ids": [seq_length],
                "input_mask": [seq_length],
                "label_ids": [],
            },
            padding_values={
                "input_ids": 0,
                "segment_ids": 0,
                "input_mask": 0,
                "label_ids": 0,
            },
            drop_remainder=True)
        return dataset
    return input_fn

In [ ]:
def main(_):
    tf.logging.set_verbosity(tf.logging.INFO)
    
    if not FLAGS.do_train and not FLAGS.do_eval:
        raise ValueError("At least one of `do_train` or `do_eval` must be True.")

    bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)

    if FLAGS.max_seq_length > bert_config.max_position_embeddings:
        raise ValueError(
            "Cannot use sequence length %d because the BERT model "
            "was only trained up to sequence length %d" %
            (FLAGS.max_seq_length, bert_config.max_position_embeddings))

    tf.gfile.MakeDirs(FLAGS.output_dir)

    tpu_cluster_resolver = None
    if FLAGS.use_tpu and FLAGS.tpu_name:
        tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
            FLAGS.tpu_name, zone=FLAGS.tpu_zone, project=FLAGS.gcp_project)
    
    is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
    run_config = tf.contrib.tpu.RunConfig(
        cluster=tpu_cluster_resolver,
        master=FLAGS.master,
        model_dir=FLAGS.output_dir,
        save_checkpoints_steps=FLAGS.save_checkpoints_steps,
        tpu_config=tf.contrib.tpu.TPUConfig(
            iterations_per_loop=FLAGS.iterations_per_loop,
            num_shards=FLAGS.num_tpu_cores,
            per_host_input_for_training=is_per_host))
    
    model_fn = model_fn_builder(
        bert_config=bert_config,
        num_labels=2,
        init_checkpoint=FLAGS.init_checkpoint,
        learning_rate=FLAGS.learning_rate,
        num_train_steps=FLAGS.num_train_steps,
        num_warmup_steps=FLAGS.num_warmup_steps,
        use_tpu=FLAGS.use_tpu,
        use_one_hot_embeddings=FLAGS.use_tpu)

  # If TPU is not available, this will fall back to normal Estimator on CPU
  # or GPU.
    estimator = tf.contrib.tpu.TPUEstimator(
        use_tpu=FLAGS.use_tpu,
        model_fn=model_fn,
        config=run_config,
        train_batch_size=FLAGS.train_batch_size,
        eval_batch_size=FLAGS.eval_batch_size,
        predict_batch_size=FLAGS.eval_batch_size)

    if FLAGS.do_train:
        tf.logging.info("***** Running training *****")
        tf.logging.info("  Batch size = %d", FLAGS.train_batch_size)
        tf.logging.info("  Num steps = %d", FLAGS.num_train_steps)
        train_input_fn = input_fn_builder(
            dataset_path=FLAGS.data_dir + "/dataset_train.tf",
            seq_length=FLAGS.max_seq_length,
            is_training=True)
        estimator.train(input_fn=train_input_fn,
                    max_steps=FLAGS.num_train_steps)
        tf.logging.info("Done Training!")

    if FLAGS.do_eval:
        for set_name in ["dev", "eval"]:
            tf.logging.info("***** Running evaluation *****")
            tf.logging.info("  Batch size = %d", FLAGS.eval_batch_size)
            max_eval_examples = None
            if FLAGS.max_eval_examples:
                max_eval_examples = FLAGS.max_eval_examples * FLAGS.num_eval_docs

            eval_input_fn = input_fn_builder(
                dataset_path=FLAGS.data_dir + "/dataset_" + set_name + ".tf",
                seq_length=FLAGS.max_seq_length,
                is_training=False,
                max_eval_examples=max_eval_examples)

            tf.logging.info("Computing metrics...")

            if FLAGS.msmarco_output:
                msmarco_file = tf.gfile.Open(
                    FLAGS.output_dir + "/msmarco_predictions_" + set_name + ".tsv", "w")
                query_docids_map = []
                with tf.gfile.Open(
                    FLAGS.data_dir + "/query_doc_ids_" + set_name + ".txt") as ref_file:
                    for line in ref_file:
                        query_docids_map.append(line.strip().split("\t"))

            result = estimator.predict(input_fn=eval_input_fn,
                                 yield_single_examples=True)
            start_time = time.time()
            results = []
            all_metrics = np.zeros(len(METRICS_MAP))
            example_idx = 0
            total_count = 0
            for item in result:
                results.append((item["log_probs"], item["label_ids"]))
                if total_count % 10000 == 0:
                    tf.logging.info("Read {} examples in {} secs".format(
                        total_count, int(time.time() - start_time)))

                if len(results) == FLAGS.num_eval_docs:
                    
                    log_probs, labels = zip(*results)
                    log_probs = np.stack(log_probs).reshape(-1, 2)
                    labels = np.stack(labels)

                    scores = log_probs[:, 1]
                    pred_docs = scores.argsort()[::-1]
                    gt = set(list(np.where(labels > 0)[0]))

                    all_metrics += metrics.metrics(
                        gt=gt, pred=pred_docs, metrics_map=METRICS_MAP)

                    if FLAGS.msmarco_output:
                        start_idx = example_idx * FLAGS.num_eval_docs
                        end_idx = (example_idx + 1) * FLAGS.num_eval_docs
                        query_ids, doc_ids = zip(*query_docids_map[start_idx:end_idx])
                        assert len(set(query_ids)) == 1, "Query ids must be all the same."
                        query_id = query_ids[0]
                        rank = 1
                        for doc_idx in pred_docs:
                            doc_id = doc_ids[doc_idx]
                            
                            if doc_id != "00000000":
                                msmarco_file.write(
                                    "\t".join((query_id, doc_id, str(rank))) + "\n")
                            rank += 1

                        example_idx += 1
                        results = []

                    total_count += 1

                if FLAGS.msmarco_output:
                    msmarco_file.close()
                    
                all_metrics /= example_idx

                tf.logging.info("Eval {}:".format(set_name))
                tf.logging.info("  ".join(METRICS_MAP))
                tf.logging.info(all_metrics)

In [ ]:
if __name__ == "__main__":
    tf.app.run()